### Preparacion de dataframes auxiliares para los endpoints

En este notebook, estaremos preparando dataframes auxiliares que contienen únicamente información correspondiente a la consulta en particular, y a su vez estaremos desarrollando y probando las funciones que serán los endpoints en la API.

#### Importamos las librerias a utilizar

In [3]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import Tools as t
import pyarrow.parquet as pq
warnings.filterwarnings("ignore")

Cargamos los datasets que ya previamente limpiados

In [4]:
games = pd.read_csv('data_ok/games_cleaned.csv', encoding='utf-8')
items = pd.read_csv('data_ok/users_items_cleaned.csv', encoding='utf-8')
reviews = pd.read_csv('data_ok/users_reviews_cleaned.csv', encoding='utf-8')

 <h2 align=center> ______________________________________________________________________<h2>

#### PlayTimeGenre. Preparamos el dataframe para la función 

##### Procesamiento

Unimos los dataframes que contienen los campos necesarios para realizar la consulta

In [5]:
df_PlayTimeGenre = pd.merge(items, games, on = 'item_id', how= 'left')  
eliminar = ['item_name','user_id','items_count','steam_id','app_name', 'developer']
df_PlayTimeGenre = df_PlayTimeGenre.drop(columns=eliminar)
df_PlayTimeGenre

,item_id,playtime_forever,genres,released_year
0,10,6.0,Action,2000.0
1,20,0.0,Action,1999.0
2,30,7.0,Action,2003.0
3,40,0.0,Action,2001.0
4,50,0.0,Action,1999.0
...,...,...,...,...
10920786,388490,3.0,Free to Play,2015.0
10920787,521570,4.0,Casual,2016.0
10920788,521570,4.0,Free to Play,2016.0
10920789,521570,4.0,Indie,2016.0


Revisamos los nulos, de ser necesario, los eliminamos

In [6]:
nulos = t.PorcentajeNulos(df_PlayTimeGenre)
nulos

,%_valores_nulos,Cantidad_Nulos,Cantidad_NO_Nulos,Total_Registros
item_id,0.00,0,10920791,10920791
playtime_forever,0.00,0,10920791,10920791
genres,8.89,970926,9949865,10920791
released_year,8.84,965111,9955680,10920791


In [7]:
df_PlayTimeGenre.dropna()

,item_id,playtime_forever,genres,released_year
0,10,6.0,Action,2000.0
1,20,0.0,Action,1999.0
2,30,7.0,Action,2003.0
3,40,0.0,Action,2001.0
4,50,0.0,Action,1999.0
...,...,...,...,...
10920785,388490,3.0,Adventure,2015.0
10920786,388490,3.0,Free to Play,2015.0
10920787,521570,4.0,Casual,2016.0
10920788,521570,4.0,Free to Play,2016.0


Buscamos los registros con playtiem_forever == 0, quiere decir que no jugaron nunca y procedemos a eliminarlos

In [8]:
df_PlayTimeGenre = df_PlayTimeGenre.loc[df_PlayTimeGenre['playtime_forever'] != 0]      
df_PlayTimeGenre 

,item_id,playtime_forever,genres,released_year
0,10,6.0,Action,2000.0
2,30,7.0,Action,2003.0
8,300,4733.0,Action,2010.0
9,240,1853.0,Action,2004.0
10,3830,333.0,Action,2005.0
...,...,...,...,...
10920786,388490,3.0,Free to Play,2015.0
10920787,521570,4.0,Casual,2016.0
10920788,521570,4.0,Free to Play,2016.0
10920789,521570,4.0,Indie,2016.0


Agrupamos por géneros y por años, y calculamos el total de horas jugadas

In [9]:
df_PlayTimeGenre = df_PlayTimeGenre.groupby([ 'genres', 'released_year'])['playtime_forever'].sum().reset_index()               # Hacemos un dataframe con las columnas necesarias ya argupadas, y suma el total de horas jugadas
df_PlayTimeGenre.rename(columns= {'playtime_forever':'Total_horas_jugadas'}, inplace = True)                                    # Renombramos las columnas
df_PlayTimeGenre.head()


,genres,released_year,Total_horas_jugadas
0,Action,1983.0,3582.0
1,Action,1984.0,384.0
2,Action,1988.0,16243.0
3,Action,1989.0,607.0
4,Action,1990.0,18787.0


Guardamos el dataframe en un parquet, esto para minimizar el peso del archivo y facilitar la consulta del API

In [11]:
df_PlayTimeGenre.to_parquet('datos_parquet/df_PlayTimeGenre.parquet')

##### Función

In [12]:
def PlayTimeGenre(df, genero:str):

    mascara = (df['genres'] == genero)                                                                                         # Filtramos el dataframe para que solo sean de el género de la consulta
    df_PlayTimeGenre_sorted = df[mascara].sort_values(by='Total_horas_jugadas', ascending=False).head(1)                       # Acomodamos el df en orden descendente en relacion a Total horas jugadas

    year = df_PlayTimeGenre_sorted['released_year'].values[0]                                                                  # Sacamos el año del género con más horas jugadas

    resultado = {f'Año de lanzamiento con más horas jugadas para el género {genero}': year}
    return resultado

Probamos la función

In [13]:
prueba = PlayTimeGenre(df_PlayTimeGenre,'Action')
print(prueba)

{'Año de lanzamiento con más horas jugadas para el género Action': 2012.0}


 <h2 align=center> ______________________________________________________________________<h2>

#### UserForGenre. 

##### Procesamiento

Unimos los dataframes que contienen los campos necesarios para realizar la consulta

In [14]:
df_UserForGenre = pd.merge(items, games, on = 'item_id', how= 'left')  
eliminar = ['item_id', 'item_name','items_count','steam_id','app_name', 'developer']
df_UserForGenre = df_UserForGenre.drop(columns=eliminar)

df_UserForGenre

,playtime_forever,user_id,genres,released_year
0,6.0,76561197970982479,Action,2000.0
1,0.0,76561197970982479,Action,1999.0
2,7.0,76561197970982479,Action,2003.0
3,0.0,76561197970982479,Action,2001.0
4,0.0,76561197970982479,Action,1999.0
...,...,...,...,...
10920786,3.0,76561198329548331,Free to Play,2015.0
10920787,4.0,76561198329548331,Casual,2016.0
10920788,4.0,76561198329548331,Free to Play,2016.0
10920789,4.0,76561198329548331,Indie,2016.0


Buscamos los registros con playtiem_forever == 0, quiere decir que no jugaron nunca y procedemos a eliminarlos

In [15]:
df_UserForGenre = df_UserForGenre.loc[df_UserForGenre['playtime_forever'] != 0]      
df_UserForGenre                  

,playtime_forever,user_id,genres,released_year
0,6.0,76561197970982479,Action,2000.0
2,7.0,76561197970982479,Action,2003.0
8,4733.0,76561197970982479,Action,2010.0
9,1853.0,76561197970982479,Action,2004.0
10,333.0,76561197970982479,Action,2005.0
...,...,...,...,...
10920786,3.0,76561198329548331,Free to Play,2015.0
10920787,4.0,76561198329548331,Casual,2016.0
10920788,4.0,76561198329548331,Free to Play,2016.0
10920789,4.0,76561198329548331,Indie,2016.0


Revisamos la cantidad de nulos que se generaron

In [16]:
nulos =  t.PorcentajeNulos(df_UserForGenre)
nulos

,%_valores_nulos,Cantidad_Nulos,Cantidad_NO_Nulos,Total_Registros
playtime_forever,0.00,0,7399876,7399876
user_id,0.00,0,7399876,7399876
genres,6.68,494161,6905715,7399876
released_year,6.64,491364,6908512,7399876


Eliminamos los nulos

In [17]:
df_UserForGenre1 = df_UserForGenre.dropna()

Agrupamos por user_id, género y año el dataframe, para sumar el total de horas jugadas

In [18]:
df_UserForGenre_group = df_UserForGenre1.groupby(['genres', 'user_id', 'released_year'])['playtime_forever'].sum().reset_index()                # Hacemos un dataframe con las columnas necesarias ya argupadas, y suma el total de horas jugadas
df_UserForGenre_group.rename(columns= {'playtime_forever':'Total_horas_jugadas'}, inplace = True)                                               # Le cambiamos el nombre a la columna

Almacenamos el dataframe en un archivo parquet

In [19]:
df_UserForGenre_group.to_parquet('datos_parquet/df_UserForGenre.parquet')

##### Función

Ya con el dataframe listo, procedemos a realizar la función 

In [20]:
def UserForGenre(df, genero: str):                     
    mascara = (df['genres'] == genero)
    df_UserForGenre = df[mascara]

    df_UserForGenre_sorted = df_UserForGenre.groupby('user_id')['Total_horas_jugadas'].sum().reset_index()                        # Obtenemos el usuario con más horas jugadas para el género
    user_max_playtime = df_UserForGenre_sorted.sort_values(by='Total_horas_jugadas', ascending=False).iloc[0]['user_id']          # Acomodamos de mayor a menor el dataframe, y tomamos el primero

    df_UserGenreUser = df_UserForGenre[df_UserForGenre['user_id'] == user_max_playtime]                                           # Filtramos los datos para ese usuario y género

    df_UserGenreUserByYear = df_UserGenreUser.groupby('released_year')['Total_horas_jugadas'].sum().reset_index()                 # Obtenemos el total de horas jugadas por año para el usuario y género seleccionados

    horas_por_anio = df_UserGenreUserByYear.to_dict(orient='records')                                                             # Creamos la lista de años y horas jugadas

    return {f"Usuario con más horas jugadas para Género {genero}": user_max_playtime,"Horas jugadas": horas_por_anio}



Probamos la función

In [21]:
resultado = UserForGenre(df_UserForGenre_group, 'Action')
print(resultado)

{'Usuario con más horas jugadas para Género Action': 'Sp3ctre', 'Horas jugadas': [{'released_year': 1995.0, 'Total_horas_jugadas': 217.0}, {'released_year': 1999.0, 'Total_horas_jugadas': 44.0}, {'released_year': 2000.0, 'Total_horas_jugadas': 70644.0}, {'released_year': 2001.0, 'Total_horas_jugadas': 13.0}, {'released_year': 2002.0, 'Total_horas_jugadas': 238.0}, {'released_year': 2003.0, 'Total_horas_jugadas': 7673.0}, {'released_year': 2004.0, 'Total_horas_jugadas': 127411.0}, {'released_year': 2005.0, 'Total_horas_jugadas': 21339.0}, {'released_year': 2006.0, 'Total_horas_jugadas': 896.0}, {'released_year': 2007.0, 'Total_horas_jugadas': 112784.0}, {'released_year': 2008.0, 'Total_horas_jugadas': 224.0}, {'released_year': 2009.0, 'Total_horas_jugadas': 108326.0}, {'released_year': 2010.0, 'Total_horas_jugadas': 78083.0}, {'released_year': 2011.0, 'Total_horas_jugadas': 154896.0}, {'released_year': 2012.0, 'Total_horas_jugadas': 378296.0}, {'released_year': 2013.0, 'Total_horas_juga

 <h2 align=center> ______________________________________________________________________<h2>

#### UsersRecommend. 

##### Procesamiento

Unimos los dataframes que contienen los campos necesarios para realizar la consulta

In [5]:
mascara = (reviews['recommend'] == 1) & ((reviews['score'] == 1) | (reviews['score'] == 2))
best_reviews = reviews[mascara]

df_best_reviews_count = best_reviews.groupby(['item_id', 'year_posted']).size().reset_index(name='Total_good_reviews')         # Obtenemos el conteo de malas reseñas por 'item_id' y 'year_posted'

df_best_reviews_2 = pd.merge(df_best_reviews_count,games)                                                                      # Unimos el dataframe creado, con el de games para obtener el título de los videojuegos
eliminar = ['genres','released_year','developer']                                                                              # Para este endpoint no son necesarias estas columnas
df_UsersRecommend = df_best_reviews_2.drop(columns=eliminar)                                                                   # Por lo que las eliminamos
df_UsersRecommend

,item_id,year_posted,Total_good_reviews,app_name
0,10,2011,1,Counter-Strike
1,10,2012,3,Counter-Strike
2,10,2013,6,Counter-Strike
3,10,2014,26,Counter-Strike
4,10,2015,15,Counter-Strike
...,...,...,...,...
10371,521570,2014,1,You Have 10 Seconds 2
10372,521990,2014,1,Galactic Storm
10373,521990,2014,1,Galactic Storm
10374,527340,2014,1,What The Box?


Guardamos el dataframe en un archivo parquet

In [24]:
df_UsersRecommend.to_parquet('datos_parquet/df_UsersRecommend.parquet')

##### Función

Hacemos la funcion UsersRecommend

In [6]:
def UsersRecommend(df_best_reviews_2,anio:int):

    mascara = (df_best_reviews_2['year_posted'] == anio)   
    df_best_reviews_3 = df_best_reviews_2[mascara]
    name_counts = df_best_reviews_3['app_name'].value_counts().head(3)
 
    resultados = []
    for puesto, (name, count) in enumerate(name_counts.items(), start=1):                            # Crear una lista de diccionarios con el formato solicitado
        resultados.append({f"Puesto {puesto}": name})

    return resultados

Probamos la función

In [7]:
resultado_bestrev = UsersRecommend(df_UsersRecommend,2014)
print(resultado_bestrev)

[{'Puesto 1': 'Miscreated'}, {'Puesto 2': 'Deepworld'}, {'Puesto 3': 'Naval Action'}]


 <h2 align=center> ______________________________________________________________________<h2>

#### UsersWorstDeveloper. Preparamos el dataframe para la funcion 

##### Procesamiento

In [8]:
mascara = (reviews['score'] == 0) & (reviews['recommend'] == 0)                                                                 # Para ser considerado malo, debe tener recommend = o y score = 0
worst_reviews = reviews[mascara]                                                                                                # Filtramos el df con estas condiciones

df_worst_reviews_count = worst_reviews.groupby(['item_id', 'year_posted']).size().reset_index(name='Total_bad_reviews')         # Obtenemos el conteo de malas reseñas por 'item_id' y 'year_posted'

df_worst_reviews_2 = pd.merge(df_worst_reviews_count,games)
eliminar = ['genres','released_year','app_name']                                                                                # Para este endpoint no son necesarias estas columnas
df_worst_reviews_2 = df_worst_reviews_2.drop(columns=eliminar)                                                                  # Por lo que las eliminamos
df_worst_reviews_2

,item_id,year_posted,Total_bad_reviews,developer
0,20,2014,1,Valve
1,80,2014,1,Valve
2,220,2014,1,Valve
3,240,2013,1,Valve
4,360,2014,1,Valve
...,...,...,...,...
1950,506140,2014,1,"Cloudcade, Inc."
1951,506140,2014,1,"Cloudcade, Inc."
1952,506140,2014,1,"Cloudcade, Inc."
1953,506140,2014,1,"Cloudcade, Inc."


In [28]:
df_worst_reviews_2.to_parquet('datos_parquet/df_UsersWorstDeveloper.parquet')

##### Función

Hacemos la funcion UsersWorstDeveloper

In [29]:
def UsersWorstDeveloper(df_worst_reviews_2,anio:int):

    mascara = (df_worst_reviews_2['year_posted'] == anio)   
    df_worst_reviews_3 = df_worst_reviews_2[mascara]
    developer_counts = df_worst_reviews_3['developer'].value_counts().head(3)
 
    resultados = []
    for puesto, (developer, count) in enumerate(developer_counts.items(), start=1):                            # Crear una lista de diccionarios con el formato solicitado
        resultados.append({f"Puesto {puesto}": developer})

    return resultados

Lo probamos

In [30]:
resultado_final = UsersWorstDeveloper(df_worst_reviews_2,2015)
print(resultado_final)

[{'Puesto 1': 'ProjectorGames'}, {'Puesto 2': 'The Behemoth'}, {'Puesto 3': 'Daybreak Game Company'}]


 <h2 align=center> ______________________________________________________________________<h2>

#### Sentiment Analysis. Preparamos el dataframe para la función

##### Procesamiento

In [9]:
df_Sentiment_An = pd.merge(games[['item_id','developer']], reviews[['item_id','score']], on ='item_id', how = 'inner')                          # Unimos el df de games (solo la columna developer) con el df reviews (solo la columna score)
df_Sentiment_An.drop(columns=['item_id'], inplace=True)                                                                                         # No necesitamos la columna item_id para esta consulta
df_Sentiment_An.head()

,developer,score
0,Stainless Games Ltd,2
1,Stainless Games Ltd,2
2,Stainless Games Ltd,2
3,Valve,0
4,Valve,2


Guardamos el dataframe en un archivo parquet para su posterior analisis

In [10]:
df_Sentiment_An.to_parquet('datos_parquet/df_SentimentAnalysis.parquet')

##### Función

In [33]:
def SentimentAnalysis(df, developer:str):
    
    mascara = (df['developer'] == developer)
    df1 = df[mascara]

    pos = df1[df1['score'] == 2]['score'].sum()
    neu = df1[df1['score'] == 1]['score'].sum()
    neg = df1[df1['score'] == 0]['score'].sum()

    final = {f'{developer}:[Negative = {neg}, Neutral = {neu}, Positive = {pos}]'}

    return final

prueba = SentimentAnalysis(df_Sentiment_An,'Stainless Games Ltd')
prueba

{'Stainless Games Ltd:[Negative = 0, Neutral = 0, Positive = 10]'}

Este es el fin de esta parte del proceso, porfavor da click [aqui](04_SistemaDeRecomendacion.ipynb) para continuar.